In [ ]:
import os
import time
import shutil
import glob
import rdflib
import pandas as pd
from ipywidgets import Button, Label, HBox
import qgrid

In [ ]:
%%html
<style>
.mytext {
    font-style: bold;
    font-size: 20px;
    padding: 25px 25px 25px 25px;
}
</style>

<div class="mytext" style="background-color:#BCBCBC;max-width: 100%">
<a href="." data-toggle="popover" title="Open Notebook Files" style="horizontal-align:middle;text-decoration:none;color:white"><i class="fa fa-file"></i>
Click Here to go to your Home Directory</a>
</div>

## Welcome to the HydroShare Notebook Server 

You have arrived at the HydroShare Notebook Server landing page. Jupyter notebook servers enable scientists to explore, modify, and interact with data inside a remote execution environment by using the Python programming language.  A **Jupyter notebook** is an enhanced computational environment that combines rich text and code execution into a single script-like container. The HydroShare Python Notebook Server aims to combine this standard notebook functionality with the HydroShare data hosting platform to provided a rich computational environment for water scientists.

To learn more:
* [QuickStart](/user-redirect/notebooks/QuickStart.ipynb)

* [Users Guide](/user-redirect/notebooks/UsersGuide.ipynb)


Hydroshare Resources you have downloaded are listed below.


In [ ]:
del_dir = ''

def select_cb(event, w):
    global del_dir
    if event['new'] == []:
        del_button.disabled=True
    else:
        if del_dir:
            # cancel confirm dialog
            hbox.children = [del_button]
            del_dir = ''
        del_button.disabled=False
    
del_button = Button (
                description='Delete Resource',
                icon='minus',
                tooltip='Remove Resource and its Files',
                button_style='danger',
                disabled=True
            )

confirm_label = Label(value='')
cancel_but = Button(description='Cancel', button_style='info')
ok_but = Button(description='OK',
                button_style='danger'
               )

confirm_dialog = [confirm_label, cancel_but, ok_but]

In [ ]:
def del_clicked(_):
    # delete clicked.  bring up confirm dialog
    global del_dir
    del_button.disabled = True
    ok_but.disabled = False
    cancel_but.disabled = False
    selected_dir = qw.get_selected_df()['Directory'][0]
    d = selected_dir.split('"')[1]
    del_dir = os.path.dirname(os.path.dirname(d))
    if del_dir.startswith('notebooks'):
        del_dir = os.path.dirname(del_dir)
    confirm_label.value = 'Delete all files in %s ?' % del_dir
    hbox.children = confirm_dialog
    #qw.change_selection([])

def del_project(_):
    # confirmed. Delete it
    
    global del_dir
    ok_but.disabled = True
    cancel_but.disabled = True
    del_button.disabled = True
    
    shutil.rmtree(del_dir)
    qw.df = build_df()
    del_dir = ''
    hbox.children = [del_button]
    qw.change_selection([])
    
def del_cancel(_):
    # canceled delete
    global del_dir
    del_dir = ''
    ok_but.disabled = True
    cancel_but.disabled = True
    del_button.disabled = True
    hbox.children = [del_button]
    qw.change_selection([])
    
ok_but.on_click(del_project)
cancel_but.on_click(del_cancel)

In [ ]:
def build_df():
    files = glob.glob('Downloads/*/data/resourcemetadata.xml')
    files.extend(glob.glob('notebooks/data/*/*/data/resourcemetadata.xml'))

    data = {'Name': [],
            'Date': [],
            'Directory': [],
            'Size (KB)': []
           }

    for f in files:
        g = rdflib.Graph()
        g.parse(f)
        title='unknown'
        for s,p,o in g:
            if p.title().endswith('/Title'):
                title = o.toPython()
                break
        _dir = os.path.dirname(f)
        contents = os.path.join(_dir, 'contents')
        content_files = glob.glob('%s/**' % contents, recursive=True)
        size = int(sum(os.path.getsize(x) for x in content_files)/1024)
        latest_file = max(content_files, key=os.path.getmtime)
        ltime = time.strftime("%Y-%m-%d", time.localtime(os.path.getmtime(latest_file)))
        data['Name'].append(title)
        data['Date'].append(ltime)
        data['Directory'].append(contents)
        data['Size (KB)'].append(size)
        g.close()
        
    # change directory names into links
    data['Directory'] = ['<a href="{}" target="_blank">{}</a>'.format(x,x) for x in data['Directory']]
    df = pd.DataFrame.from_dict(data)
    return df.set_index('Name')

In [ ]:
hbox = HBox([del_button])
df = build_df()
qw = qgrid.show_grid(df, grid_options={'editable': False})
display(qw)
display(hbox)
qw.on('selection_changed', select_cb)
del_button.on_click(del_clicked)